<a href="https://colab.research.google.com/github/sharma18b/Image-Captioning/blob/main/Colab%20Works/image_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import cv2
import os
import numpy as np
from keras.models import load_model, Model
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
def get_model(layer='fc2'):
    """Keras Model of the VGG16 network, with the output layer set to `layer`.
    The default layer is the second-to-last fully connected layer 'fc2' of
    shape (4096,).
    Parameters
    ----------
    layer : str
        which layer to extract (must be of shape (None, X)), e.g. 'fc2', 'fc1'
        or 'flatten'
    """
    # base_model.summary():
    #     ....
    #     block5_conv4 (Conv2D)        (None, 15, 15, 512)       2359808
    #     _________________________________________________________________
    #     block5_pool (MaxPooling2D)   (None, 7, 7, 512)         0
    #     _________________________________________________________________
    #     flatten (Flatten)            (None, 25088)             0
    #     _________________________________________________________________
    #     fc1 (Dense)                  (None, 4096)              102764544
    #     _________________________________________________________________
    #     fc2 (Dense)                  (None, 4096)              16781312
    #     _________________________________________________________________
    #     predictions (Dense)          (None, 1000)              4097000
    #
    base_model = VGG16(weights='imagenet', include_top=True)
    model = Model(inputs=base_model.input,
                outputs=base_model.get_layer(layer).output)
    return model

In [ ]:
def get_files(path_to_files, size):
    fn_imgs = []
    coun =0
    files = [file for file in os.listdir(path_to_files)]
    for file in files:
        img = cv2.resize(cv2.imread(path_to_files+file), size)
        fn_imgs.append([file, img])
        coun += 1
        if coun==1000:
          break
    return dict(fn_imgs)

In [ ]:

def feature_vector(img_arr, model):
    if img_arr.shape[2] == 1:
        img_arr = img_arr.repeat(3, axis=2)

    # (1, 224, 224, 3)
    arr4d = np.expand_dims(img_arr, axis=0)  
    arr4d_pp = preprocess_input(arr4d)
    return model.predict(arr4d_pp)[0,:]

In [ ]:
def feature_vectors(imgs_dict, model):
    f_vect = {}
    for fn, img in imgs_dict.items():
        f_vect[fn] = feature_vector(img, model)
    return f_vect

In [ ]:
imgs_dict = get_files(path_to_files = '/content/drive/MyDrive/kaggle_dataset/flickr8k-sau/Flickr_Data/Images/',size = (224, 224))

In [ ]:
# Create Keras NN model.
model = get_model()

# Feed images through the model and extract feature vectors.
img_feature_vector = feature_vectors(imgs_dict, model)

553467904/553467096 [==============================] - 6s 0us/step


In [ ]:
images = list(img_feature_vector.values())
kmeans = KMeans(n_clusters=8, init='k-means++')
kmeans.fit(images)
y_kmeans = kmeans.predict(images)
file_names = list(imgs_dict.keys())

In [ ]:
n_clusters = 9
cluster_path = '/content/drive/MyDrive/kaggle_dataset/flickr8k-sau/Flickr_Data/'
path_to_files = '/content/drive/MyDrive/kaggle_dataset/flickr8k-sau/Flickr_Data/Images/'

for c in range(0,n_clusters):
    if not os.path.exists(cluster_path+'cluster_'+str(c)):
        os.mkdir(cluster_path+'cluster_'+str(c))
    
for fn, cluster in zip(file_names, y_kmeans):
    image = cv2.imread(path_to_files+fn)
    cv2.imwrite(cluster_path+'cluster_'+str(cluster)+'/'+fn, image)

In [ ]:
print(img_feature_vector)

{'380034515_4fbdfa6b26.jpg': array([8.02691  , 2.8501906, 0.       , ..., 0.       , 4.012636 ,
       0.       ], dtype=float32), '383223174_7165a54c30.jpg': array([0.83184624, 0.        , 0.        , ..., 0.        , 0.60240537,
       3.0811625 ], dtype=float32), '3726168984_1fa2c8965b.jpg': array([0.47985798, 0.        , 0.        , ..., 1.6090391 , 0.0386256 ,
       1.3324945 ], dtype=float32), '384465575_31294122c0.jpg': array([2.1202374, 0.       , 0.       , ..., 0.       , 0.1651366,
       1.6414247], dtype=float32), '378453580_21d688748e.jpg': array([4.611166, 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
      dtype=float32), '391324644_d23fdf06cb.jpg': array([0.       , 2.4572453, 0.2763899, ..., 1.4275365, 0.       ,
       1.5888174], dtype=float32), '3728015645_b43a60258b.jpg': array([0.        , 1.4320298 , 0.1010589 , ..., 0.11525479, 0.        ,
       0.        ], dtype=float32), '394563330_68b566368c.jpg': array([5.6441584, 0.       , 0.       , ..., 0. 

In [ ]:
print(len(file_names))
print(len(img_feature_vector))
print(y_kmeans)

1000
1000
[2 0 2 5 2 5 7 7 0 7 7 3 1 6 4 7 7 0 7 5 6 6 4 3 7 3 2 2 7 1 2 2 5 0 2 3 0
 7 1 1 4 1 7 7 3 5 2 7 4 3 5 1 4 3 1 1 4 1 1 4 5 1 7 3 7 3 7 3 1 2 7 0 4 7
 4 6 6 3 4 5 1 1 7 7 1 0 2 1 3 0 6 2 7 7 6 2 6 6 6 2 5 5 4 5 2 6 4 7 0 0 2
 2 7 5 6 2 2 0 2 3 7 0 4 0 0 6 7 3 2 6 3 5 1 4 3 2 0 1 6 6 4 7 4 7 7 2 2 7
 7 6 0 7 6 7 3 5 2 0 7 6 0 3 7 3 4 2 2 6 5 5 7 6 3 5 3 2 0 5 6 2 3 2 0 3 2
 2 2 2 0 0 3 2 6 2 7 1 4 4 1 6 7 4 6 0 7 0 7 5 0 2 2 2 4 2 6 7 3 7 6 6 7 3
 6 0 0 7 0 2 2 2 2 6 2 6 2 7 5 1 2 7 4 3 7 1 2 1 5 6 5 1 2 0 2 6 7 7 4 2 7
 6 1 3 5 4 1 2 2 7 4 0 7 1 0 1 2 2 7 2 5 5 5 5 2 1 0 4 2 3 6 0 0 3 5 7 7 5
 6 5 7 2 6 6 0 4 4 5 5 4 1 2 2 7 2 0 1 7 3 2 2 2 1 5 0 6 7 0 2 0 2 1 4 7 4
 4 5 0 4 4 5 5 0 0 1 7 0 7 0 2 7 4 2 2 7 2 4 7 0 1 1 5 5 0 0 3 3 0 1 4 7 5
 1 5 4 3 5 7 0 7 7 4 1 3 7 7 1 4 1 0 4 0 2 2 2 3 4 7 0 0 5 1 5 0 7 1 1 0 5
 2 6 2 6 4 2 4 2 7 1 5 6 5 1 0 7 4 3 7 0 6 0 3 4 2 0 5 0 2 3 2 6 7 3 2 4 7
 5 7 6 3 2 2 6 1 1 0 3 5 2 4 0 7 1 1 1 2 6 4 7 4 7 3 7 5 2 2 7 3 1 0 4 4 4
 3 6 7 1 5 2 2 

In [ ]:
print(len(img_feature_vector['743571049_68080e8751.jpg']))

4096
